In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("nlp_example").getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [3]:
from pyspark.sql.functions import col, udf

In [4]:
from pyspark.sql.types import IntegerType

In [5]:
sen_df = spark.createDataFrame([(0, "Hi I heard about Spark"),
                               (1, "I wish java could use case classes"),
                               (2, "Logistic,regression,models,are,neat")],
                              ["id", "sentence"])

In [6]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [9]:
regex_tokenizer = RegexTokenizer(inputCol="sentence", outputCol="words",
                                pattern="\\W")

In [10]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [11]:
tokenized = tokenizer.transform(sen_df)

In [13]:
tokenized.withColumn("tokens", count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [14]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [15]:
rg_tokenized.withColumn("tokens", count_tokens(col("words"))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [16]:
from pyspark.ml.feature import StopWordsRemover

In [17]:
sentenceDataFrame = spark.createDataFrame([(0, ["I", "saw", "a", "green", "horse"]),
                               (1, ["Mary", "had", "a", "little", "lamb"]),],
                              ["id", "tokens"])

In [18]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, a, green...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [19]:
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

In [20]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, a, green...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [21]:
from pyspark.ml.feature import NGram

In [22]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (0, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (0, ["Logistic", "regression", "models", "are", "neat"])],
    ["id", "words"])

In [23]:
ngram = NGram(n=2, inputCol="words", outputCol="grams")

In [27]:
ngram.transform(wordDataFrame).select("grams").show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

